1. Derive the analytical gradients for the model parameters given the square loss.

$$                   \mathcal{L} = \frac{1}{n}\displaystyle\sum_{i=1}{\frac{(y_i -\hat{y_i})^2}{2}}
$$

Let $$ \hat{y_i} = \sigma(z_i) - sigmoid function $$
where $$z_i = \sum_{j=1}^{m} x_{ij}.w_j + b$$

By applying the chain rule to get the gradients:

$$\frac{\delta \mathcal L}{\delta w_j} = \sum_{i=1}^{n}\frac{\delta \mathcal L}{\delta \hat{y}_i} \times \frac{\delta \hat{y}_i}{\delta z_i} \times \frac{\delta z_i}{\delta w_j} \hspace{1cm}(1) $$
$$\frac{\delta \mathcal L}{\delta b} = \sum_{i=1}^{n}\frac{\delta \mathcal L}{\delta \hat{y}_i} \times \frac{\delta \hat{y}_i}{\delta z_i} \times \frac{\delta z_i}{\delta b} \hspace{1.5cm}  (2) $$


(1) Since we know that:

$$ \sigma'(z_i) = \sigma(z_i) (1 - \sigma(z_i))$$
then
$$ \frac{\delta \hat{y}_i}{\delta z_i}  = \hat{y_i}(1 - \hat{y_i})$$

Also
$$ \frac{\delta \mathcal L}{\delta \hat{y}_i} = \hat{y_i} - {y_i} $$
and 
  $$ \frac{\delta z_i}{\delta w_j} = x_{ij}$$
  
As a result,
$$\frac{\delta \mathcal L}{\delta w_j} = \sum_{i=1}^{n}(\hat{y_i} - {y_i})\hat{y_i}(1 - \hat{y_i})x_{ij} $$
   


For (2) in addition, we calculate:
$$\frac{\delta z_i}{\delta b} = 1$$

Then
$$\frac{\delta \mathcal L}{\delta b} = \sum_{i=1}^{n}(\hat{y_i} - {y_i})\hat{y_i}(1 - \hat{y_i}) $$

2. Provide code verifying that your analytical gradients are correct using finite difference for
two or more samples from the training data (you are free to use an external package that
calculates the finite difference).